Link drive untuk keseluruhan program dan weight dari model
https://drive.google.com/drive/folders/1n7aADZefyso8p4TqZvdtimTNwBFPkFNc?usp=sharing

# Util

In [ ]:
class IdMap:
    """
    Ingat kembali di kuliah, bahwa secara praktis, sebuah dokumen dan
    sebuah term akan direpresentasikan sebagai sebuah integer. Oleh
    karena itu, kita perlu maintain mapping antara string term (atau
    dokumen) ke integer yang bersesuaian, dan sebaliknya. Kelas IdMap ini
    akan melakukan hal tersebut.
    """

    def __init__(self):
        """
        Mapping dari string (term atau nama dokumen) ke id disimpan dalam
        python's dictionary; cukup efisien. Mapping sebaliknya disimpan dalam
        python's list.

        contoh:
            str_to_id["halo"] ---> 8
            str_to_id["/collection/dir0/gamma.txt"] ---> 54

            id_to_str[8] ---> "halo"
            id_to_str[54] ---> "/collection/dir0/gamma.txt"
        """
        self.str_to_id = {}
        self.id_to_str = []

    def __len__(self):
        """Mengembalikan banyaknya term (atau dokumen) yang disimpan di IdMap."""
        return len(self.id_to_str)

    def __get_str(self, i):
        """Mengembalikan string yang terasosiasi dengan index i."""
        # TODO
        return self.id_to_str[i]

    def __get_id(self, s):
        """
        Mengembalikan integer id i yang berkorespondensi dengan sebuah string s.
        Jika s tidak ada pada IdMap, lalu assign sebuah integer id baru dan kembalikan
        integer id baru tersebut.
        """
        # TODO
        if (s not in self.id_to_str):
            self.id_to_str.append(s)
            self.str_to_id[s] = len(self.id_to_str) - 1
        return self.str_to_id[s]

    def __getitem__(self, key):
        """
        __getitem__(...) adalah special method di Python, yang mengizinkan sebuah
        collection class (seperti IdMap ini) mempunyai mekanisme akses atau
        modifikasi elemen dengan syntax [..] seperti pada list dan dictionary di Python.

        Silakan search informasi ini di Web search engine favorit Anda. Saya mendapatkan
        link berikut:

        https://stackoverflow.com/questions/43627405/understanding-getitem-method

        Jika key adalah integer, gunakan __get_str;
        jika key adalah string, gunakan __get_id
        """
        if type(key) is int:
            return self.__get_str(key)
        elif type(key) is str:
            return self.__get_id(key)
        else:
            raise TypeError

def sorted_merge_posts_and_tfs(posts_tfs1, posts_tfs2):
    """
    Menggabung (merge) dua lists of tuples (doc id, tf) dan mengembalikan
    hasil penggabungan keduanya (TF perlu diakumulasikan untuk semua tuple
    dengn doc id yang sama), dengan aturan berikut:

    contoh: posts_tfs1 = [(1, 34), (3, 2), (4, 23)]
            posts_tfs2 = [(1, 11), (2, 4), (4, 3 ), (6, 13)]

            return   [(1, 34+11), (2, 4), (3, 2), (4, 23+3), (6, 13)]
                   = [(1, 45), (2, 4), (3, 2), (4, 26), (6, 13)]

    Parameters
    ----------
    list1: List[(Comparable, int)]
    list2: List[(Comparable, int]
        Dua buah sorted list of tuples yang akan di-merge.

    Returns
    -------
    List[(Comparablem, int)]
        Penggabungan yang sudah terurut
    """
    result = []
    i = j = 0
    while i < len(posts_tfs1) and j < len(posts_tfs2):
        if (posts_tfs1[i][0] == posts_tfs2[j][0]):
            result.append((posts_tfs1[i][0], posts_tfs1[i][1] + posts_tfs2[j][1]))
            i += 1; j+= 1
        elif (posts_tfs1[i][0] < posts_tfs2[j][0]):
            result.append(posts_tfs1[i])
            i += 1
        elif (posts_tfs1[i][0] > posts_tfs2[j][0]):
            result.append(posts_tfs2[j])
            j += 1
    while (i < len(posts_tfs1)):
        result.append(posts_tfs1[i])
        i += 1
    while (j < len(posts_tfs2)):
        result.append(posts_tfs2[j])
        j += 1
    return result

# Compression

In [ ]:
import array
class StandardPostings:
    """ 
    Class dengan static methods, untuk mengubah representasi postings list
    yang awalnya adalah List of integer, berubah menjadi sequence of bytes.
    Kita menggunakan Library array di Python.

    ASUMSI: postings_list untuk sebuah term MUAT di memori!

    Silakan pelajari:
        https://docs.python.org/3/library/array.html
    """

    @staticmethod
    def encode(postings_list):
        """
        Encode postings_list menjadi stream of bytes

        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)

        Returns
        -------
        bytes
            bytearray yang merepresentasikan urutan integer di postings_list
        """
        # Untuk yang standard, gunakan L untuk unsigned long, karena docID
        # tidak akan negatif. Dan kita asumsikan docID yang paling besar
        # cukup ditampung di representasi 4 byte unsigned.
        return array.array('L', postings_list).tobytes()

    @staticmethod
    def decode(encoded_postings_list):
        """
        Decodes postings_list dari sebuah stream of bytes

        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray merepresentasikan encoded postings list sebagai keluaran
            dari static method encode di atas.

        Returns
        -------
        List[int]
            list of docIDs yang merupakan hasil decoding dari encoded_postings_list
        """
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

    @staticmethod
    def encode_tf(tf_list):
        """
        Encode list of term frequencies menjadi stream of bytes

        Parameters
        ----------
        tf_list: List[int]
            List of term frequencies

        Returns
        -------
        bytes
            bytearray yang merepresentasikan nilai raw TF kemunculan term di setiap
            dokumen pada list of postings
        """
        return StandardPostings.encode(tf_list)

    @staticmethod
    def decode_tf(encoded_tf_list):
        """
        Decodes list of term frequencies dari sebuah stream of bytes

        Parameters
        ----------
        encoded_tf_list: bytes
            bytearray merepresentasikan encoded term frequencies list sebagai keluaran
            dari static method encode_tf di atas.

        Returns
        -------
        List[int]
            List of term frequencies yang merupakan hasil decoding dari encoded_tf_list
        """
        return StandardPostings.decode(encoded_tf_list)

class VBEPostings:
    """ 
    Berbeda dengan StandardPostings, dimana untuk suatu postings list,
    yang disimpan di disk adalah sequence of integers asli dari postings
    list tersebut apa adanya.

    Pada VBEPostings, kali ini, yang disimpan adalah gap-nya, kecuali
    posting yang pertama. Barulah setelah itu di-encode dengan Variable-Byte
    Enconding algorithm ke bytestream.

    Contoh:
    postings list [34, 67, 89, 454] akan diubah dulu menjadi gap-based,
    yaitu [34, 33, 22, 365]. Barulah setelah itu di-encode dengan algoritma
    compression Variable-Byte Encoding, dan kemudian diubah ke bytesream.

    ASUMSI: postings_list untuk sebuah term MUAT di memori!

    """

    @staticmethod
    def vb_encode_number(number):
        """
        Encodes a number using Variable-Byte Encoding
        Lihat buku teks kita!
        """
        bytes = []
        while True:
            bytes.insert(0, number % 128) # prepend ke depan
            if number < 128:
                break
            number = number // 128
        bytes[-1] += 128 # bit awal pada byte terakhir diganti 1
        return array.array('B', bytes).tobytes()

    @staticmethod
    def vb_encode(list_of_numbers):
        """ 
        Melakukan encoding (tentunya dengan compression) terhadap
        list of numbers, dengan Variable-Byte Encoding
        """
        bytes = []
        for number in list_of_numbers:
            bytes.append(VBEPostings.vb_encode_number(number))
        return b"".join(bytes)

    @staticmethod
    def encode(postings_list):
        """
        Encode postings_list menjadi stream of bytes (dengan Variable-Byte
        Encoding). JANGAN LUPA diubah dulu ke gap-based list, sebelum
        di-encode dan diubah ke bytearray.

        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)

        Returns
        -------
        bytes
            bytearray yang merepresentasikan urutan integer di postings_list
        """
        diff = [postings_list[0] if postings_list != [] else None]
        for i in range(1, len(postings_list)):
            diff.append(postings_list[i] - postings_list[i-1])
        return VBEPostings.vb_encode(diff)

    @staticmethod
    def encode_tf(tf_list):
        """
        Encode list of term frequencies menjadi stream of bytes

        Parameters
        ----------
        tf_list: List[int]
            List of term frequencies

        Returns
        -------
        bytes
            bytearray yang merepresentasikan nilai raw TF kemunculan term di setiap
            dokumen pada list of postings
        """
        return VBEPostings.vb_encode(tf_list)

    @staticmethod
    def vb_decode(encoded_bytestream):
        """
        Decoding sebuah bytestream yang sebelumnya di-encode dengan
        variable-byte encoding.
        """
        # TODO
        return None

    @staticmethod
    def vb_decode(encoded_bytestream):
        """
        Decoding sebuah bytestream yang sebelumnya di-encode dengan
        variable-byte encoding.
        """
        numbers = []
        n = 0
        for byte in encoded_bytestream:
            if (byte < 128):
                n = 128 * n + byte
            else:
                n = 128 * n + byte - 128
                numbers.append(n)
                n = 0
        return numbers

    @staticmethod
    def decode(encoded_postings_list):
        """
        Decodes postings_list dari sebuah stream of bytes. JANGAN LUPA
        bytestream yang di-decode dari encoded_postings_list masih berupa
        gap-based list.

        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray merepresentasikan encoded postings list sebagai keluaran
            dari static method encode di atas.

        Returns
        -------
        List[int]
            list of docIDs yang merupakan hasil decoding dari encoded_postings_list
        """
        diff = VBEPostings.vb_decode(encoded_postings_list)
        decoded = [diff[0] if diff != [] else None]
        for i in range(1, len(diff)):
            decoded.append(diff[i] + decoded[i-1])
        return decoded

    @staticmethod
    def decode_tf(encoded_tf_list):
        """
        Decodes list of term frequencies dari sebuah stream of bytes

        Parameters
        ----------
        encoded_tf_list: bytes
            bytearray merepresentasikan encoded term frequencies list sebagai keluaran
            dari static method encode_tf di atas.

        Returns
        -------
        List[int]
            List of term frequencies yang merupakan hasil decoding dari encoded_tf_list
        """
        return VBEPostings.vb_decode(encoded_tf_list)

# Index

In [ ]:
import pickle
import os
class InvertedIndex:
    """
    Class yang mengimplementasikan bagaimana caranya scan atau membaca secara
    efisien Inverted Index yang disimpan di sebuah file; dan juga menyediakan
    mekanisme untuk menulis Inverted Index ke file (storage) saat melakukan indexing.

    Attributes
    ----------
    postings_dict: Dictionary mapping:

            termID -> (start_position_in_index_file,
                       number_of_postings_in_list,
                       length_in_bytes_of_postings_list,
                       length_in_bytes_of_tf_list)

        postings_dict adalah konsep "Dictionary" yang merupakan bagian dari
        Inverted Index. postings_dict ini diasumsikan dapat dimuat semuanya
        di memori.

        Seperti namanya, "Dictionary" diimplementasikan sebagai python's Dictionary
        yang memetakan term ID (integer) ke 4-tuple:
           1. start_position_in_index_file : (dalam satuan bytes) posisi dimana
              postings yang bersesuaian berada di file (storage). Kita bisa
              menggunakan operasi "seek" untuk mencapainya.
           2. number_of_postings_in_list : berapa banyak docID yang ada pada
              postings (Document Frequency)
           3. length_in_bytes_of_postings_list : panjang postings list dalam
              satuan byte.
           4. length_in_bytes_of_tf_list : panjang list of term frequencies dari
              postings list terkait dalam satuan byte

    terms: List[int]
        List of terms IDs, untuk mengingat urutan terms yang dimasukan ke
        dalam Inverted Index.

    """
    def __init__(self, index_name, postings_encoding, directory=''):
        """
        Parameters
        ----------
        index_name (str): Nama yang digunakan untuk menyimpan files yang berisi index
        postings_encoding : Lihat di compression.py, kandidatnya adalah StandardPostings,
                        GapBasedPostings, dsb.
        directory (str): directory dimana file index berada
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         # Untuk keep track urutan term yang dimasukkan ke index
        self.doc_length = {}    # key: doc ID (int), value: document length (number of tokens)
                                # Ini nantinya akan berguna untuk normalisasi Score terhadap panjang
                                # dokumen saat menghitung score dengan TF-IDF atau BM25
        self.doc_length_average = 0

    def __enter__(self):
        """
        Memuat semua metadata ketika memasuki context.
        Metadata:
            1. Dictionary ---> postings_dict
            2. iterator untuk List yang berisi urutan term yang masuk ke
                index saat konstruksi. ---> term_iter
            3. doc_length, sebuah python's dictionary yang berisi key = doc id, dan
                value berupa banyaknya token dalam dokumen tersebut (panjang dokumen).
                Berguna untuk normalisasi panjang saat menggunakan TF-IDF atau BM25
                scoring regime; berguna untuk untuk mengetahui nilai N saat hitung IDF,
                dimana N adalah banyaknya dokumen di koleksi

        Metadata disimpan ke file dengan bantuan library "pickle"

        Perlu memahani juga special method __enter__(..) pada Python dan juga
        konsep Context Manager di Python. Silakan pelajari link berikut:

        https://docs.python.org/3/reference/datamodel.html#object.__enter__
        """
        # Membuka index file
        if (not os.path.isfile(self.index_file_path)):
          open(self.index_file_path, 'w+')
        self.index_file = open(self.index_file_path, 'rb+')

        # Kita muat postings dict dan terms iterator dari file metadata
        if (not os.path.isfile(self.metadata_file_path)):
          open(self.metadata_file_path, 'w+')
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms, self.doc_length, self.doc_length_average = pickle.load(f)
            self.term_iter = self.terms.__iter__()

        return self

    def __exit__(self, exception_type, exception_value, traceback):
        """Menutup index_file dan menyimpan postings_dict dan terms ketika keluar context"""
        # Menutup index file
        self.index_file.close()
        self.doc_length_average = sum(self.doc_length.values())/len(self.doc_length.values())
        # Menyimpan metadata (postings dict dan terms) ke file metadata dengan bantuan pickle
        with open(self.metadata_file_path, 'wb') as f:
            pickle.dump([self.postings_dict, self.terms, self.doc_length, self.doc_length_average], f)


class InvertedIndexReader(InvertedIndex):
    """
    Class yang mengimplementasikan bagaimana caranya scan atau membaca secara
    efisien Inverted Index yang disimpan di sebuah file.
    """
    def __iter__(self):
        return self

    def reset(self):
        """
        Kembalikan file pointer ke awal, dan kembalikan pointer iterator
        term ke awal
        """
        self.index_file.seek(0)
        self.term_iter = self.terms.__iter__() # reset term iterator

    def __next__(self): 
        """
        Class InvertedIndexReader juga bersifat iterable (mempunyai iterator).
        Silakan pelajari:
        https://stackoverflow.com/questions/19151/how-to-build-a-basic-iterator

        Ketika instance dari kelas InvertedIndexReader ini digunakan
        sebagai iterator pada sebuah loop scheme, special method __next__(...)
        bertugas untuk mengembalikan pasangan (term, postings_list, tf_list) berikutnya
        pada inverted index.

        PERHATIAN! method ini harus mengembalikan sebagian kecil data dari
        file index yang besar. Mengapa hanya sebagian kecil? karena agar muat
        diproses di memori. JANGAN MEMUAT SEMUA INDEX DI MEMORI!
        """
        curr_term = next(self.term_iter)
        pos, number_of_postings, len_in_bytes_of_postings, len_in_bytes_of_tf = self.postings_dict[curr_term]
        postings_list = self.postings_encoding.decode(self.index_file.read(len_in_bytes_of_postings))
        tf_list = self.postings_encoding.decode_tf(self.index_file.read(len_in_bytes_of_tf))
        return (curr_term, postings_list, tf_list)

    def get_postings_list(self, term):
        """
        Kembalikan sebuah postings list (list of docIDs) beserta list
        of term frequencies terkait untuk sebuah term (disimpan dalam
        bentuk tuple (postings_list, tf_list)).

        PERHATIAN! method tidak boleh iterasi di keseluruhan index
        dari awal hingga akhir. Method ini harus langsung loncat ke posisi
        byte tertentu pada file (index file) dimana postings list (dan juga
        list of TF) dari term disimpan.
        """
        # TODO
        if term in self.postings_dict:
            metadata = self.postings_dict[term]
            self.index_file.seek(metadata[0])
            posting_byte = self.index_file.read(metadata[2])
            tf_byte = self.index_file.read(metadata[3])
            decoded_posting = self.postings_encoding.decode(posting_byte)
            decoded_tf = self.postings_encoding.decode_tf(tf_byte)
            decoded = list(zip(decoded_posting, decoded_tf))
        else:
            decoded = []
        return decoded


class InvertedIndexWriter(InvertedIndex):
    """
    Class yang mengimplementasikan bagaimana caranya menulis secara
    efisien Inverted Index yang disimpan di sebuah file.
    """
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')
        return self

    def append(self, term, postings_list, tf_list):
        """
        Menambahkan (append) sebuah term, postings_list, dan juga TF list 
        yang terasosiasi ke posisi akhir index file.

        Method ini melakukan 4 hal:
        1. Encode postings_list menggunakan self.postings_encoding (method encode),
        2. Encode tf_list menggunakan self.postings_encoding (method encode_tf),
        3. Menyimpan metadata dalam bentuk self.terms, self.postings_dict, dan self.doc_length.
           Ingat kembali bahwa self.postings_dict memetakan sebuah termID ke
           sebuah 4-tuple: - start_position_in_index_file
                           - number_of_postings_in_list
                           - length_in_bytes_of_postings_list
                           - length_in_bytes_of_tf_list
        4. Menambahkan (append) bystream dari postings_list yang sudah di-encode dan
           tf_list yang sudah di-encode ke posisi akhir index file di harddisk.

        Jangan lupa update self.terms dan self.doc_length juga ya!

        SEARCH ON YOUR FAVORITE SEARCH ENGINE:
        - Anda mungkin mau membaca tentang Python I/O
          https://docs.python.org/3/tutorial/inputoutput.html
          Di link ini juga bisa kita pelajari bagaimana menambahkan informasi
          ke bagian akhir file.
        - Beberapa method dari object file yang mungkin berguna seperti seek(...)
          dan tell()

        Parameters
        ----------
        term:
            term atau termID yang merupakan unique identifier dari sebuah term
        postings_list: List[Int]
            List of docIDs dimana term muncul
        tf_list: List[Int]
            List of term frequencies
        """
        # TODO
        
        encode_posting = self.postings_encoding.encode(postings_list)
        encode_tf = self.postings_encoding.encode_tf(tf_list)
        self.postings_dict[term] = (self.index_file.tell(), len(postings_list), len(encode_posting), len(encode_tf))
        self.index_file.write(encode_posting)
        self.index_file.write(encode_tf)
        self.terms.append(term)
        for i in range(len(postings_list)):
            if (postings_list[i] in self.doc_length):
                self.doc_length[postings_list[i]] = self.doc_length[postings_list[i]] + tf_list[i]
            else:
                self.doc_length[postings_list[i]] = tf_list[i]

# BSBI

In [ ]:
!mkdir index

In [ ]:
import os
import pickle
import contextlib
import heapq
import time
import math

from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
class BSBIIndex:
    """
    Attributes
    ----------
    term_id_map(IdMap): Untuk mapping terms ke termIDs
    doc_id_map(IdMap): Untuk mapping relative paths dari dokumen (misal,
                    /collection/0/gamma.txt) to docIDs
    data_dir(str): Path ke data
    output_dir(str): Path ke output index files
    postings_encoding: Lihat di compression.py, kandidatnya adalah StandardPostings,
                    VBEPostings, dsb.
    index_name(str): Nama dari file yang berisi inverted index
    """
    def __init__(self, data_dir, output_dir, postings_encoding, index_name = "main_index"):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding
        self.stemmer = SnowballStemmer(language = "english")
        # Untuk menyimpan nama-nama file dari semua intermediate inverted index
        self.intermediate_indices = []
        self.load()

    def save(self):
        """Menyimpan doc_id_map and term_id_map ke output directory via pickle"""
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pickle.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pickle.dump(self.doc_id_map, f)

    def load(self):
        """Memuat doc_id_map and term_id_map dari output directory"""
        try:
            with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
                self.term_id_map = pickle.load(f)
            with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
                self.doc_id_map = pickle.load(f)
        except:
            print("Create File")

    def clean_text(self, text):
        text = text.lower()
        text = re.sub("\s+", " ", text) # Menghilangkan spasi berlebih
        text = re.sub("[^\w\s]", " ", text) # Menghilangkan tanda baca'
        stop_words = set(stopwords.words('english'))
        res = word_tokenize(text)
        res = [w for w in res if w not in stop_words]
        return [self.stemmer.stem(w) for w in res]

    def parse_block(self, block_dir_relative):
        """
        Lakukan parsing terhadap text file sehingga menjadi sequence of
        <termID, docID> pairs.

        Gunakan tools available untuk Stemming Bahasa Inggris

        JANGAN LUPA BUANG STOPWORDS!

        Untuk "sentence segmentation" dan "tokenization", bisa menggunakan
        regex atau boleh juga menggunakan tools lain yang berbasis machine
        learning.

        Parameters
        ----------
        block_dir_relative : str
            Relative Path ke directory yang mengandung text files untuk sebuah block.

            CATAT bahwa satu folder di collection dianggap merepresentasikan satu block.
            Konsep block di soal tugas ini berbeda dengan konsep block yang terkait
            dengan operating systems.

        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
            Mengembalikan semua pasangan <termID, docID> dari sebuah block (dalam hal
            ini sebuah sub-direktori di dalam folder collection)

        Harus menggunakan self.term_id_map dan self.doc_id_map untuk mendapatkan
        termIDs dan docIDs. Dua variable ini harus 'persist' untuk semua pemanggilan
        parse_block(...).
        """
        # TODO
        collection_path = os.path.join(self.data_dir, block_dir_relative)
        td_pairs = []
        for filename in os.listdir(collection_path):
            with open(os.path.join(collection_path, filename), 'r') as f: # open in readonly mode
                doc_id = self.doc_id_map[os.path.join(collection_path, filename)]
                text = f.read()
                tokens = self.clean_text(text)
                for token in tokens:
                    term_id = self.term_id_map[token]
                    td_pairs.append((term_id, doc_id))
        return td_pairs
    
    def invert_write(self, td_pairs, index):
        """
        Melakukan inversion td_pairs (list of <termID, docID> pairs) dan
        menyimpan mereka ke index. Disini diterapkan konsep BSBI dimana 
        hanya di-mantain satu dictionary besar untuk keseluruhan block.
        Namun dalam teknik penyimpanannya digunakan srategi dari SPIMI
        yaitu penggunaan struktur data hashtable (dalam Python bisa
        berupa Dictionary)

        ASUMSI: td_pairs CUKUP di memori

        Di Tugas Pemrograman 1, kita hanya menambahkan term dan
        juga list of sorted Doc IDs. Sekarang di Tugas Pemrograman 2,
        kita juga perlu tambahkan list of TF.

        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index pada disk (file) yang terkait dengan suatu "block"
        """
        term_dict = {}
        for term_id, doc_id in td_pairs:
            if term_id not in term_dict:
                term_dict[term_id] = {}
            if (doc_id not in term_dict[term_id].keys()):
                term_dict[term_id][doc_id] = 0
            term_dict[term_id][doc_id] +=1
        for term_id in sorted(term_dict.keys()):
            posting_list = sorted(list(term_dict[term_id]))
            tf_list = []
            for i in posting_list:
                tf_list.append(term_dict[term_id][i])
            index.append(term_id, posting_list, tf_list)
        
        

    def merge(self, indices, merged_index):
        """
        Lakukan merging ke semua intermediate inverted indices menjadi
        sebuah single index.

        Ini adalah bagian yang melakukan EXTERNAL MERGE SORT

        Gunakan fungsi orted_merge_posts_and_tfs(..) di modul util

        Parameters
        ----------
        indices: List[InvertedIndexReader]
            A list of intermediate InvertedIndexReader objects, masing-masing
            merepresentasikan sebuah intermediate inveted index yang iterable
            di sebuah block.

        merged_index: InvertedIndexWriter
            Instance InvertedIndexWriter object yang merupakan hasil merging dari
            semua intermediate InvertedIndexWriter objects.
        """
        # kode berikut mengasumsikan minimal ada 1 term
        merged_iter = heapq.merge(*indices, key = lambda x: x[0])
        curr, postings, tf_list = next(merged_iter) # first item
        for t, postings_, tf_list_ in merged_iter: # from the second item
            if t == curr:
                zip_p_tf = sorted_merge_posts_and_tfs(list(zip(postings, tf_list)), \
                                                      list(zip(postings_, tf_list_)))
                postings = [doc_id for (doc_id, _) in zip_p_tf]
                tf_list = [tf for (_, tf) in zip_p_tf]
            else:
                merged_index.append(curr, postings, tf_list)
                curr, postings, tf_list = t, postings_, tf_list_
        merged_index.append(curr, postings, tf_list)

    def bm25_okapi(self, tf, df, N, dl, avdl,k1 = 1.2, b=0.75):
        """
        Menghitung score dokumen dengan metode BM25 tanpa normalisasi panjang dokumen.

        Parameters
        ----------
        tf: int
            Term frequency 
        df: int
            Document frequency
        N:  int
            Total document
        k1: float
            Fine tuning
        """
        return math.log10(N/df) * ((k1+1) * tf)/(k1 * ((1-b) + b * (dl/avdl)) + tf)

    def tf_n(self, tf):
        return tf
    def tf_l(self, tf):
        return (1 + math.log10(tf)) if tf > 0 else 0
    def tf_b(self, tf):
        return 1 if tf > 0 else 0
    def df_n(self, df, N):
        return 1
    def df_t(self, df, N):
        return (math.log10(N/df))
    def df_p(self, df, N):
        return max(0, math.log10((N-df)/df))

    def tfidf(self, notation, raw_tf, raw_df, N):
        """
        Menghitung score dokumen dengan metode BM25 tanpa normalisasi panjang dokumen.

        Parameters
        ----------
        notation: string
            Smart notation for tf-idf
        tf: int
            Term frequency 
        df: int
            Document frequency
        N:  int
            Total document
        """
        tf = 0
        df = 0
        if (notation[0] == "n"):
            tf = self.tf_n(raw_tf)
        elif (notation[0] == "l"):
            tf = self.tf_l(raw_tf)
        elif (notation[0] == "b"):
            tf = self.tf_b(raw_tf)
        if (notation[1] == "n"):
            df = self.df_n(raw_df, N)
        elif (notation[1] == "t"):
            df = self.df_t(raw_df, N)
        elif (notation[1] == "p"):
            df = self.df_p(raw_df, N)
        return tf * df

    def retrieve_tfidf(self, query, k = 10, notation = "ltn"):
        """
        Melakukan Ranked Retrieval dengan skema TaaT (Term-at-a-Time).
        Method akan mengembalikan top-K retrieval results.

        w(t, D) = (1 + log tf(t, D))       jika tf(t, D) > 0
                = 0                        jika sebaliknya

        w(t, Q) = IDF = log (N / df(t))

        Score = untuk setiap term di query, akumulasikan w(t, Q) * w(t, D).
                (tidak perlu dinormalisasi dengan panjang dokumen)

        catatan: 
            1. informasi DF(t) ada di dictionary postings_dict pada merged index
            2. informasi TF(t, D) ada di tf_li
            3. informasi N bisa didapat dari doc_length pada merged index, len(doc_length)

        Parameters
        ----------
        query: str
            Query tokens yang dipisahkan oleh spasi

            contoh: Query "universitas indonesia depok" artinya ada
            tiga terms: universitas, indonesia, dan depok

        Result
        ------
        List[(int, str)]
            List of tuple: elemen pertama adalah score similarity, dan yang
            kedua adalah nama dokumen.
            Daftar Top-K dokumen terurut mengecil BERDASARKAN SKOR.

        JANGAN LEMPAR ERROR/EXCEPTION untuk terms yang TIDAK ADA di collection.

        """
        # TODO
        tokens = self.clean_text(query)
        page_dict = {}
        result = []
        with InvertedIndexReader(self.index_name, self.postings_encoding, directory = self.output_dir) as merged_index:
            for token in tokens:
                term_id = self.term_id_map[token]
                postings = merged_index.get_postings_list(term_id)
                for (doc_id, tf) in postings: 
                    doc_str = self.doc_id_map[doc_id]
                    if (doc_str not in page_dict):
                        page_dict[doc_str] = 0
                    page_dict[doc_str] += self.tfidf(notation, tf, merged_index.postings_dict[doc_id][1], len(self.doc_id_map))
            result = list(zip(page_dict.values(), page_dict.keys()))
        result = sorted(result, key=lambda x: x[0], reverse=True)
        return result[:k]

    def retrieve_bm25(self, query, k = 10, k1 = 1.2, b = 0.75):
        """
        Melakukan Ranked Retrieval dengan skema TaaT (Term-at-a-Time).
        Method akan mengembalikan top-K retrieval results.

        w(t, D) = (1 + log tf(t, D))       jika tf(t, D) > 0
                = 0                        jika sebaliknya

        w(t, Q) = IDF = log (N / df(t))

        Score = untuk setiap term di query, akumulasikan w(t, Q) * w(t, D).
                (tidak perlu dinormalisasi dengan panjang dokumen)

        catatan: 
            1. informasi DF(t) ada di dictionary postings_dict pada merged index
            2. informasi TF(t, D) ada di tf_li
            3. informasi N bisa didapat dari doc_length pada merged index, len(doc_length)

        Parameters
        ----------
        query: str
            Query tokens yang dipisahkan oleh spasi

            contoh: Query "universitas indonesia depok" artinya ada
            tiga terms: universitas, indonesia, dan depok

        Result
        ------
        List[(int, str)]
            List of tuple: elemen pertama adalah score similarity, dan yang
            kedua adalah nama dokumen.
            Daftar Top-K dokumen terurut mengecil BERDASARKAN SKOR.

        JANGAN LEMPAR ERROR/EXCEPTION untuk terms yang TIDAK ADA di collection.

        """
        # TODO
        tokens = self.clean_text(query)
        page_dict = {}
        result = []
        with InvertedIndexReader(self.index_name, self.postings_encoding, directory = self.output_dir) as merged_index:
            for token in tokens:
                term_id = self.term_id_map[token]
                postings = merged_index.get_postings_list(term_id)
                for (doc_id, tf) in postings: 
                    doc_str = self.doc_id_map[doc_id]
                    if (doc_str not in page_dict):
                        page_dict[doc_str] = 0
                    page_dict[doc_str] += self.bm25_okapi(tf, merged_index.postings_dict[doc_id][1], len(self.doc_id_map), \
                        merged_index.doc_length[doc_id], merged_index.doc_length_average, k1, b)
            result = list(zip(page_dict.values(), page_dict.keys()))
        result = sorted(result, key=lambda x: x[0], reverse=True)
        return result[:k]

    def index(self):
        """
        Base indexing code
        BAGIAN UTAMA untuk melakukan Indexing dengan skema BSBI (blocked-sort
        based indexing)

        Method ini scan terhadap semua data di collection, memanggil parse_block
        untuk parsing dokumen dan memanggil invert_write yang melakukan inversion
        di setiap block dan menyimpannya ke index yang baru.
        """
        # loop untuk setiap sub-directory di dalam folder collection (setiap block)
        for block_dir_relative in tqdm(sorted(next(os.walk(self.data_dir))[1])):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'intermediate_index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, self.postings_encoding, directory = self.output_dir) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, self.postings_encoding, directory = self.output_dir) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(InvertedIndexReader(index_id, self.postings_encoding, directory=self.output_dir))
                               for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
BSBI_instance = BSBIIndex(data_dir = '/content/drive/MyDrive/IR/collection', \
                              postings_encoding = VBEPostings, \
                              output_dir = 'index')
BSBI_instance.index() # memulai indexing!


Create File


100%|██████████| 11/11 [01:48<00:00,  9.89s/it]


# Mono Roberta

In [ ]:
from IPython.display import clear_output
!pip install transformers
clear_output()

In [ ]:
from transformers import TFRobertaModel
from keras.layers import Dropout, Dense
from keras.optimizers import Adam
import tensorflow as tf
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from transformers import RobertaTokenizer
class MonoRoberta(tf.keras.Model):

    def __init__(self, model_name, dropout_prob=0.1):
        super().__init__(name="reranker")
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.roberta = TFRobertaModel.from_pretrained(model_name, from_pt=True)
        self.dropout = Dropout(dropout_prob)
        weight_initializer = tf.keras.initializers.GlorotNormal() 
        self.classifier = Dense(3, name="classifier", 
                                # activation = 'softmax',
                                kernel_initializer = weight_initializer,  
                                bias_initializer = 'zeros')

    def call(self, inputs, **kwargs):
        trained_roberta = self.roberta(inputs, **kwargs)
        pooled_output = trained_roberta.pooler_output
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        logits = self.classifier(pooled_output)
        return logits
    

# DPR Roberta

In [ ]:
from transformers import TFRobertaModel
from keras.layers import Dropout, Dense, Dot
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

class DPRRoberta(tf.keras.Model):

    def __init__(self, model_name, dropout_prob=0.3):
        super().__init__(name="reranker")
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.roberta_query = TFRobertaModel.from_pretrained(model_name, from_pt=True)
        self.roberta_doc = TFRobertaModel.from_pretrained(model_name, from_pt=True)
        self.dropout = Dropout(dropout_prob)
        self.dot = Dot(axes=1)


    def call(self, query, doc, **kwargs):
        trained_query = self.roberta_query(query, **kwargs)
        trained_doc= self.roberta_doc(doc, **kwargs)
        pooled_query = trained_query.pooler_output
        pooled_doc = trained_doc.pooler_output
        return self.dot([pooled_query, pooled_doc])

# Letor

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
class Letor:
    def reranking(self, query, docs_path):
        docs = []
        for doc_path in docs_path:
            docs.append((doc_path[1], open(doc_path[1], "r").read()))
        scores = self.predict(query, docs)
        did_scores = [x for x in zip(scores, [did for (did, _) in docs])]
        # return did_scores
        return sorted(did_scores, key = lambda tup: tup[0], reverse = True)

In [ ]:
class LetorMono(Letor):
    def __init__(self, model_name):
        self.model = MonoRoberta(model_name)
    
    def prepare_model(self, model_path):
        encoded_input = self.model.tokenizer("a", "b", padding=True, truncation=True, max_length=256, return_tensors='tf')
        self.model(encoded_input)
        self.model.load_weights(model_path)

    def predict(self, query, docs):
          scores = []
          for doc in docs:
              x = self.model.tokenizer(query, doc[1], padding="max_length", truncation=True, max_length=256, return_tensors='tf')
              out = self.model(x).numpy()[0]
              scores.append(out[2])
          return scores


In [ ]:
class LetorDPR(Letor):
    def __init__(self, model_name):
        self.model = DPRRoberta(model_name)
    
    def prepare_model(self, model_path):
        x_q = self.model.tokenizer("a", padding="max_length", truncation=True, max_length=50, return_tensors='tf')
        x_d = self.model.tokenizer("b", padding="max_length", truncation=True, max_length=206, return_tensors='tf')
        self.model(x_q, x_d)
        self.model.load_weights(model_path)

    def predict(self, query, docs):
          scores = []
          for doc in docs:
              x_q = self.model.tokenizer(query, padding="max_length", truncation=True, max_length=50, return_tensors='tf')
              x_d = self.model.tokenizer(doc[1], padding="max_length", truncation=True, max_length=206, return_tensors='tf')
              scores.append(self.model(x_q, x_d).numpy()[0][0])
          return scores


In [ ]:
BSBI_instance = BSBIIndex(data_dir = '/content/drive/MyDrive/IR/collection', \
                      postings_encoding = VBEPostings, \
                      output_dir = 'index')
query = "alkylated"
bsbi = BSBI_instance.retrieve_bm25(query, k = 10, k1=1.2, b=0.75)
print("Query  : ", query)
print("Results:")
for (score, doc) in bsbi:
    print(f"{doc:30} {score:>.3f}")
print()
letor_instance = LetorMono("allenai/biomed_roberta_base")
letor_instance.prepare_model("/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")
letor = letor_instance.reranking(query, bsbi)
print("Results :")
for (score, did) in letor:
    print(did, score)

Query  :  alkylated
Results:
/content/drive/MyDrive/IR/collection/10/989.txt 2.549
/content/drive/MyDrive/IR/collection/6/547.txt 2.373
/content/drive/MyDrive/IR/collection/11/1003.txt 1.921
/content/drive/MyDrive/IR/collection/6/507.txt 1.829
/content/drive/MyDrive/IR/collection/8/785.txt 1.813
/content/drive/MyDrive/IR/collection/5/482.txt 1.727



Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/656M [00:00<?, ?B/s]

Results :
/content/drive/MyDrive/IR/collection/6/507.txt 1.1304115
/content/drive/MyDrive/IR/collection/11/1003.txt 0.70857996
/content/drive/MyDrive/IR/collection/6/547.txt 0.6443668
/content/drive/MyDrive/IR/collection/8/785.txt 0.48496473
/content/drive/MyDrive/IR/collection/5/482.txt 0.4375329
/content/drive/MyDrive/IR/collection/10/989.txt 0.24341339


# Eksperimen

In [ ]:
import math
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

######## >>>>> 3 IR metrics: RBP p = 0.8, DCG, dan AP
qr_count = {'Q1': 37, 'Q2': 16, 'Q3': 22, 'Q4': 23, 'Q5': 26, 'Q6': 13, 'Q7': 15, 'Q8': 11, 'Q9': 28, 'Q10': 24, 'Q11': 18, 'Q12': 9, 'Q13': 21, 'Q14': 16, 'Q15': 29, 'Q16': 13, 'Q17': 21, 'Q18': 15, 'Q19': 27, 'Q20': 39, 'Q21': 27, 'Q22': 25, 'Q23': 39, 'Q24': 22, 'Q25': 24, 'Q26': 28, 'Q27': 18, 'Q28': 39, 'Q29': 37, 'Q30': 14}
def rbp(ranking, p = 0.8):
  """ menghitung search effectiveness metric score dengan 
      Rank Biased Precision (RBP)

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score RBP
  """
  score = 0.
  for i in range(1, len(ranking) + 1):
    pos = i - 1
    score += ranking[pos] * (p ** (i - 1))
  return (1 - p) * score

def irbp(ranking, key, p = 0.8):
  """ menghitung ideal RBP ketika semua ranking merupakan qrels.

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score RBP
  """
  score = 0.
  for i in range(1, len(ranking) + 1):
    rank = 1 if i <= qr_count[key] else 0
    pos = i - 1
    score += rank * (p ** (i - 1))
  return (1 - p) * score

def dcg(ranking):
  """ menghitung search effectiveness metric score dengan 
      Discounted Cumulative Gain

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score DCG
  """
  score = 0
  for i in range(len(ranking)):
    score += (1/math.log2(i+2)) * ranking[i]
  return score

def idcg(ranking, key):
  """ menghitung ideal DCG ketika semua ranking merupakan qrels.

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score DCG
  """
  score = 0
  for i in range(len(ranking)):
    rank = 1 if i < qr_count[key] else 0
    score += (1/math.log2(i+2)) * rank
  return score

def prec(ranking, k):
  score = 0
  for i in range(1, k+1):
    score += (1/k) * ranking[i-1] 
  return score 

def iprec(ranking, k, key):
  score = 0
  for i in range(1, k+1):
    rank = 1 if i <= qr_count[key] else 0
    score += (1/k) * rank
  return score 

def ap(ranking):
  """ menghitung search effectiveness metric score dengan 
      Average Precision

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score AP
  """
  # TODO
  score = 0
  R = len(list(filter(lambda x: x == 1, ranking)))
  for i in range(len(ranking)):
    if (ranking[i]):
      score += prec(ranking, i+1)/R
  return score

def iap(ranking, key):
  """ menghitung ideal AP ketika semua ranking merupakan qrels.

      Parameters
      ----------
      ranking: List[int]
         vektor biner seperti [1, 0, 1, 1, 1, 0]
         gold standard relevansi dari dokumen di rank 1, 2, 3, dst.
         Contoh: [1, 0, 1, 1, 1, 0] berarti dokumen di rank-1 relevan,
                 di rank-2 tidak relevan, di rank-3,4,5 relevan, dan
                 di rank-6 tidak relevan
        
      Returns
      -------
      Float
        score AP
  """
  # TODO
  score = 0
  R = min(qr_count[key], len(ranking))
  for i in range(len(ranking)):
    rank = 1 if i <= qr_count[key] else 0
    if (rank):
      score += iprec(ranking, i+1, key)/R
  return score
######## >>>>> memuat qrels

def load_qrels(qrel_file = "/content/drive/MyDrive/IR/qrels.txt", max_q_id = 30, max_doc_id = 1033):
  """ memuat query relevance judgment (qrels) 
      dalam format dictionary of dictionary
      qrels[query id][document id]

      dimana, misal, qrels["Q3"][12] = 1 artinya Doc 12
      relevan dengan Q3; dan qrels["Q3"][10] = 0 artinya
      Doc 10 tidak relevan dengan Q3.

  """
  qrels = {"Q" + str(i) : {i:0 for i in range(1, max_doc_id + 1)} \
                 for i in range(1, max_q_id + 1)}
  with open(qrel_file) as file:
    for line in file:
      parts = line.strip().split()
      qid = parts[0]
      did = int(parts[1])
      qrels[qid][did] = 1
  return qrels


######## >>>>> EVALUASI !

def eval_bm25(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')

  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for qline in file:
      parts = qline.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      for (score, doc) in BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b):
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))

  print("Hasil evaluasi BM25 terhadap 30 queries dengan k1 {k1} dan b {b}".format(k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

def eval_bm25_ideal_qrels(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')

  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for qline in file:
      parts = qline.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      for (score, doc) in BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b):
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
      rbp_scores.append(irbp(ranking, qid))
      dcg_scores.append(idcg(ranking, qid))
      ap_scores.append(iap(ranking, qid))

  print("Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 {k1} dan b {b}".format(k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

def eval_bm25_ideal_serp(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')

  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for qline in file:
      parts = qline.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      for (score, doc) in BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b):
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
          ranking = sorted(ranking, reverse=True)
      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))

  print("Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top {k} terhadap 30 queries dengan k1 {k1} dan b {b}".format(k=k, k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

def eval_letor_mono(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/final.h5"):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')
  # letor_instance = LetorDPR("allenai/biomed_roberta_base")
  letor_instance = LetorMono("allenai/biomed_roberta_base")
  letor_instance.prepare_model(model_path)
  num_lines = sum(1 for line in open(query_file, 'r'))
  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for line in tqdm(file, total=num_lines):
      parts = line.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      bsbi = BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b)
      letor = letor_instance.reranking(query, bsbi)
      for (score, doc) in letor:
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))
  print("Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 {k1} dan b {b}".format(k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

def eval_letor_mono(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/final.h5"):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')
  # letor_instance = LetorDPR("allenai/biomed_roberta_base")
  letor_instance = LetorMono("allenai/biomed_roberta_base")
  letor_instance.prepare_model(model_path)
  num_lines = sum(1 for line in open(query_file, 'r'))
  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for line in tqdm(file, total=num_lines):
      parts = line.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      bsbi = BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b)
      letor = letor_instance.reranking(query, bsbi)
      for (score, doc) in letor:
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))
  print("Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 {k1} dan b {b}".format(k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

def eval_letor_dpr(qrels, query_file = "/content/drive/MyDrive/IR/queries.txt", k = 1000, k1 = 1.2, b = 0.75, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/final.h5"):
  """ 
    loop ke semua 30 query, hitung score di setiap query,
    lalu hitung MEAN SCORE over those 30 queries.
    untuk setiap query, kembalikan top-1000 documents
  """
  BSBI_instance = BSBIIndex(data_dir = 'collection', \
                          postings_encoding = VBEPostings, \
                          output_dir = 'index')
  letor_instance = LetorDPR("allenai/biomed_roberta_base")
  # letor_instance = LetorMono("allenai/biomed_roberta_base")
  letor_instance.prepare_model(model_path)
  num_lines = sum(1 for line in open(query_file, 'r'))
  with open(query_file) as file:
    rbp_scores = []
    dcg_scores = []
    ap_scores = []
    for line in tqdm(file, total=num_lines):
      parts = line.strip().split()
      qid = parts[0]
      query = " ".join(parts[1:])

      # HATI-HATI, doc id saat indexing bisa jadi berbeda dengan doc id
      # yang tertera di qrels
      ranking = []
      bsbi = BSBI_instance.retrieve_bm25(query, k = k, k1=k1, b=b)
      letor = letor_instance.reranking(query, bsbi)
      for (score, doc) in letor:
          did = int(re.search(r'.*\/(\d*)\.txt', doc).group(1))
          ranking.append(qrels[qid][did])
      rbp_scores.append(rbp(ranking))
      dcg_scores.append(dcg(ranking))
      ap_scores.append(ap(ranking))
  print()
  print("Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 {k1} dan b {b}".format(k1=k1, b=b))
  print("RBP score =", sum(rbp_scores) / len(rbp_scores))
  print("DCG score =", sum(dcg_scores) / len(dcg_scores))
  print("AP score  =", sum(ap_scores) / len(ap_scores))

# Main

## Metode
Dalam percobaan ini, saya menggunakan 3 metode yang berbeda. Yaitu 2 varian Mono Roberta dan 1 DPR Roberta. Semua model Roberta yang digunakan menggunakan model pretrained [biomed_roberta_base](https://huggingface.co/allenai/biomed_roberta_base). Untuk dataset tetap menggunakan nfcorpus namun hanya 3 fitur yang dipakai. Yaitu nontopic-titles, vid-titles, dan vid-desc. Dikarenakan data terlalu besar dan keterbatasan waktu, maka untuk 1 epoch akan menggunakan 1000 data yang dipilih secara acak dari keseluruhan dataset. Berikut penjelasan mengenai model yang digunakan,

- Mono Roberta A   
Model ini menggunakan 1 buah Roberta dimana output vektor [CLS] akan dimasukkan ke dalam layer dense berukuran 3 karena menggunakan qrels dengan range relevan 1-3. Nilai skor dari query dan dokumen menggunakan nilai vektor dari indeks ke-2 atau nilai dari kelas 3 (paling relevan)
Pada saat training, saya menggabungkan nontopic-titles, vid-titles, dan vid-desc menjadi 1 query panjang. Intuisi dari metode ini agar model dapat mempelajari keseluruhan query yang digunakan untuk suatu dokumen. Setelah itu query tersebut dan dokumen akan ditokenisasi dengan maksimal panjang tokenisasi 256.
Akurasi terbaik didapatkan pada epoch ke 20.

- [Mono Roberta B](https://colab.research.google.com/drive/1QmbqsE5X2i1quAVI1QT929O8E4c23js5?usp=sharing)
Arsitektur dan cara skoring ini sama seperti Mono Roberta A. Namun terdapat perbedaan pada training dataset. Pada varian ini nontopic-titles, vid-titles, dan vid-desc tidak digabung menjadi 1 query panjang. Namun akan dipilih secara acak fitur yang digunakan saat dilakukan training. Intuisi dari metode ini karena query yang diberikan oleh user hanyalah sebuah string pertanyaan saja. Sehingga dengan 1 fitur diharapkan dapat mencerminkan sifat user dalam membuat query. Namun untuk metode ini menggunakan tokenisasi dengan maksimal panjang tokenisasi 320.
Akurasi terbaik didapatkan pada epoch ke 23.

Link colab untuk training Mono Roberta A sudah tertimpa oleh Mono Roberta B. Namun untuk proses pembuatan model sebagian besar sama. Perbedaan hanya terdapat pada penggabungan query menjadi 1. 

- [DPR Roberta](https://colab.research.google.com/drive/1WwRzZG6HDe9Et-WssGa3tVw01A2Y0swF?usp=sharing)
Untuk metode terakhir menggunakan teknik DPR dimana membutuhkan 2 model Roberta. Yaitu masing-masing query dan dokumen memiliki model Roberta tersendiri. Lalu hasil skoring didapatkan dengan mengalikan dot product dari vektor output token [CLS] query dan dokumen. Permasalahan dari model ini menjadi regresi sehingga menggunakan mean squared error (MSE) sebagai nilai loss. 
Dikarenakan query dan dokumen menggunakan model Roberta yang berbeda, maka memiliki panjang tokenisasi yang berbeda. Query memiliki panjang tokenisasi 64 dan dokumen memiliki panjang tokenisasi 256.
Akurasi terbaik didapatkan pada epoch ke 15.


## K = 2

In [ ]:
qrels = load_qrels()

In [ ]:
eval_bm25_ideal_serp(qrels, k=2)
eval_bm25_ideal_qrels(qrels, k=2)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 2 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.23866666666666672
DCG score = 1.1031625352381107
AP score  = 0.7666666666666667
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.36
DCG score = 1.6309297535714575
AP score  = 1.0


In [ ]:
k = 2
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 2
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.23600000000000007
DCG score = 1.0785578521428747
AP score  = 0.7333333333333333
Mono Roberta A


100%|██████████| 30/30 [00:09<00:00,  3.05it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.23600000000000007
DCG score = 1.0785578521428747
AP score  = 0.7333333333333333
Mono Roberta B


100%|██████████| 30/30 [00:10<00:00,  2.93it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.23466666666666672
DCG score = 1.0662555105952565
AP score  = 0.7166666666666667
DPR Roberta


100%|██████████| 30/30 [00:20<00:00,  1.47it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.2333333333333334
DCG score = 1.0539531690476385
AP score  = 0.7


## K = 3

In [ ]:
eval_bm25_ideal_serp(qrels, k=3)
eval_bm25_ideal_qrels(qrels, k=3)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 3 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.3413333333333333
DCG score = 1.5416508275000205
AP score  = 0.9333333333333333
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.48799999999999966
DCG score = 2.1309297535714578
AP score  = 1.0


In [ ]:
k = 3
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 3
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.32133333333333336
DCG score = 1.411891185476208
AP score  = 0.7777777777777778
Mono Roberta A


100%|██████████| 30/30 [00:17<00:00,  1.72it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.3306666666666666
DCG score = 1.4706198357143052
AP score  = 0.8555555555555555
Mono Roberta B


100%|██████████| 30/30 [00:14<00:00,  2.01it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.3296
DCG score = 1.4662555105952566
AP score  = 0.8583333333333332
DPR Roberta


100%|██████████| 30/30 [00:28<00:00,  1.04it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.3258666666666667
DCG score = 1.4372865023809718
AP score  = 0.8194444444444444


## K = 5

In [ ]:
eval_bm25_ideal_serp(qrels, k=5)
eval_bm25_ideal_qrels(qrels, k=5)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 5 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.4784319999999999
DCG score = 2.1054509089838547
AP score  = 0.9666666666666667
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.6723199999999995
DCG score = 2.9484591188793936
AP score  = 1.0


In [ ]:
k = 5
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 5
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.4257813333333333
DCG score = 1.8751578258173707
AP score  = 0.7676388888888889
Mono Roberta A


100%|██████████| 30/30 [00:24<00:00,  1.24it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.4517866666666666
DCG score = 2.0117121794656967
AP score  = 0.8887962962962963
Mono Roberta B


100%|██████████| 30/30 [00:27<00:00,  1.09it/s]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.4494186666666666
DCG score = 1.985664754640669
AP score  = 0.8502314814814815
DPR Roberta


100%|██████████| 30/30 [00:47<00:00,  1.57s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.44077866666666665
DCG score = 1.9389812082741313
AP score  = 0.8178240740740742


## K = 10

In [ ]:
eval_bm25_ideal_serp(qrels, k=10)
eval_bm25_ideal_qrels(qrels, k=10)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 10 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.6454066858666666
DCG score = 2.9499056054458275
AP score  = 1.0
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.8917310327466664
DCG score = 4.533923843877749
AP score  = 1.0033333333333332


In [ ]:
k = 10
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 10
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5286920772266667
DCG score = 2.5792831014735467
AP score  = 0.7439320252792475
Mono Roberta A


100%|██████████| 30/30 [00:47<00:00,  1.57s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5766538581333334
DCG score = 2.728070097955343
AP score  = 0.8274681699840428
Mono Roberta B


100%|██████████| 30/30 [00:49<00:00,  1.64s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5620415044266666
DCG score = 2.706259755024765
AP score  = 0.8073017027798773
DPR Roberta


100%|██████████| 30/30 [01:35<00:00,  3.19s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5227503104
DCG score = 2.5676963110969977
AP score  = 0.7313684964726631


## K = 30

In [ ]:
eval_bm25_ideal_serp(qrels, k=30)
eval_bm25_ideal_qrels(qrels, k=30)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 30 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.8273087802401464
DCG score = 4.368781060462851
AP score  = 1.0
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.9797155110789503
DCG score = 7.379722945036105
AP score  = 1.0429841878797386


In [ ]:
k = 30
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 30
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5619454442314002
DCG score = 3.6618154281564204
AP score  = 0.6322334163778885
Mono Roberta A


100%|██████████| 30/30 [02:22<00:00,  4.74s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.6484293110148571
DCG score = 3.9004836636710016
AP score  = 0.7479991040305471
Mono Roberta B


100%|██████████| 30/30 [02:22<00:00,  4.76s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5837128860562546
DCG score = 3.7259496108123495
AP score  = 0.6670046040019276
DPR Roberta


100%|██████████| 30/30 [04:47<00:00,  9.57s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.3963464757471192
DCG score = 3.176610615311709
AP score  = 0.4843128212608849


## K = 100

In [ ]:
eval_bm25_ideal_serp(qrels)
eval_bm25_ideal_qrels(qrels)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 1000 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.9700789167583873
DCG score = 7.152670197701833
AP score  = 0.9999999999999999
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.9798521851946951
DCG score = 7.586956031097538
AP score  = 1.0464052405113176


In [ ]:
k = 100
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 100
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5621172735609207
DCG score = 4.681040335265117
AP score  = 0.5025600764415022
Mono Roberta A


100%|██████████| 30/30 [07:30<00:00, 15.02s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.6700296381578121
DCG score = 5.142652782024445
AP score  = 0.6832292078228343
Mono Roberta B


100%|██████████| 30/30 [07:26<00:00, 14.89s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.5755932596724512
DCG score = 4.806462918663125
AP score  = 0.5688943828235057
DPR Roberta


100%|██████████| 30/30 [14:37<00:00, 29.26s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.21767954431055989
DCG score = 3.5776180705960416
AP score  = 0.28350286137729136


## K = 1000 (Default)

In [ ]:
eval_bm25_ideal_serp(qrels)
eval_bm25_ideal_qrels(qrels)

Hasil evaluasi BM25 Ideal ketika semua yang relevant ada di top 1000 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.9700789167583873
DCG score = 7.152670197701833
AP score  = 0.9999999999999999
Hasil evaluasi BM25 Ideal jika semua qrels didapatkan pada 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.9798521851946951
DCG score = 7.586956031097538
AP score  = 1.0464052405113176


In [ ]:
k = 1000
print(f"Untuk K = {k}")
print("Raw BM25")
eval_bm25(qrels, k=k)
print("Mono Roberta A")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/20.h5")  
print("Mono Roberta B")
eval_letor_mono(qrels, k=k, model_path="/content/drive/MyDrive/IR/mono roberta/checkpoint/v2_23_0.785.h5")  
print("DPR Roberta")
eval_letor_dpr(qrels, k=k, model_path="/content/drive/MyDrive/IR/DPR Roberta/checkpoint/v2_15_0.571.h5")  

Untuk K = 1000
Raw BM25
Hasil evaluasi BM25 terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.562117273564048
DCG score = 5.368178925799611
AP score  = 0.42314371344987417
Mono Roberta A


100%|██████████| 30/30 [33:12<00:00, 66.41s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.6668407389547821
DCG score = 5.981620666841301
AP score  = 0.6028828197790805
Mono Roberta B


100%|██████████| 30/30 [33:24<00:00, 66.81s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.4983931526352883
DCG score = 5.2228195438225935
AP score  = 0.4353467138778974
DPR Roberta


100%|██████████| 30/30 [1:07:28<00:00, 134.94s/it]


Hasil evaluasi BM25 Letor terhadap 30 queries dengan k1 1.2 dan b 0.75
RBP score = 0.12115606875671989
DCG score = 3.3960874335042224
AP score  = 0.14008696228875447


# Kesimpulan

Kesimpulan yang saya dapatkan bahwa metode Mono Roberta A lebih baik dibandingkan Mono Roberta B maupun DPR Roberta. Untuk k pada range 2 - 5, ketiga metode masih lebih baik dibandingkan BM25 tanpa reranking. Namun pada k bernilai 10, DPR Roberta mendapatkan hasil yang sedikit lebih buruk dari pada BM25 saja. Hal ini disebabkan karena epoch yang kurang lama karena untuk 1 epoch membutuhkan waktu yg cukup lama untuk melatihnya. Sedangkan untuk Mono ROberta varian A dan B masih memberikan hasil yang lebih baik hingga k = 100. Untuk k = 1000, hanya Mono Roberta A saja yang menghasilkan hasil yang lebih baik dibandingkan hanya BM25. Sehingga dapat disimpulkan bahwa ketika semua query digabung pada  proses training, model akan lebih bisa mempelajari maksud dari query sehingga menghasilkan akurasi yang lebih baik. Hasil Mono Roberta A juga lebih baik dibandingkan lgbm ranker pada TP 3. Sehingga membuktikan attention pada transformer dapat menangkap konteks pada kalimat lebih baik dibandingkan LSI.